In [1]:
# core libraries
import pickle
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings("ignore")
from datetime import datetime


from queries import final_features
from score_funcs import Score
sc = Score()


/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning
/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/Code


In [2]:
os.chdir('../')
cwd = os.getcwd()
print(cwd)
 
os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

from config.config import SQLQuery
querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning
/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/Code


#### Data pull for scoring 

In [3]:
df = pd.read_pickle(f'/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/data/dataset_21.pkl')

In [4]:
t = df[['business_id','emi_no'] + sc.features]

In [5]:
df.head()

,business_id,due_date,DPD_plus_15_next,DPD_plus_3_last_2,DPD_plus_15_prev,emi_no,DPD_plus_3,count_pd_deposit_1m,count_ach_debit_1m,count_ach_credit_1m,...,ratio_amt_cashin_mom,ratio_count_drawn_mom,ratio_count_cashin_mom,pilot_flag,pilot_period_flag,rs2_bin,rs2_score,fico_score,rs1_score,rs1_bin
0,c616b39b-aef7-4156-973a-08278aa74536,2023-04-03,0.0,0.0,0.0,1,0,0.0,1.0,0.0,...,2884.000000,0.25,2.000000,0,0,5,971.0,749.0,966.614760,5
1,cc861415-fd34-4654-8b97-dab0794938dc,2023-04-03,0.0,0.0,0.0,1,0,0.0,14.0,4.0,...,2433.330000,0.50,2.000000,0,0,5,822.0,670.0,886.233904,5
2,4d7e9b73-dd94-4fdc-9611-51ec641b8e19,2023-04-03,0.0,1.0,0.0,1,1,0.0,1.0,0.0,...,2196.830000,0.00,2.000000,0,0,5,850.0,722.0,810.794650,5
3,2e3c32e2-1f1b-4d0f-a7c2-8dcdb11c6317,2023-04-03,0.0,0.0,0.0,1,0,3.0,19.0,21.0,...,0.949472,0.00,0.833333,0,0,5,795.0,721.0,908.033142,5
4,57948378-e103-4f19-972e-a911082a16c2,2023-04-03,0.0,0.0,0.0,1,0,17.0,2.0,19.0,...,363.000000,0.00,1.000000,0,0,3,551.0,652.0,712.696282,5


In [6]:
df.shape

(4925, 127)

In [7]:
data_params = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/models/data_params_v2.pkl')
model = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/models/redec_3.pkl')
transformer = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/models/scale_train_object.pkl')

In [ ]:
sc.features

In [ ]:
df['emi_no'].value_counts()

In [8]:
df0 = sc.preprocess(df,sc.features,data_params,transformer)

In [9]:
df0 = sc.model_scoring_redec(df0,sc.features,model)

In [10]:
df0.head()

,business_id,due_date,DPD_plus_15_next,DPD_plus_3_last_2,DPD_plus_15_prev,emi_no,DPD_plus_3,count_pd_deposit_1m,count_ach_debit_1m,count_ach_credit_1m,...,pilot_flag,pilot_period_flag,rs2_bin,rs2_score,fico_score,rs1_score,rs1_bin,redec_proba,redec_score,redec_bin
0,c616b39b-aef7-4156-973a-08278aa74536,2023-04-03,0.0,0.0,0.0,1,0,0.0,1.0,0.0,...,0,0,5,971.0,749.0,966.614760,5,0.031045,968.955142,5
1,cc861415-fd34-4654-8b97-dab0794938dc,2023-04-03,0.0,0.0,0.0,1,0,0.0,14.0,4.0,...,0,0,5,822.0,670.0,886.233904,5,0.154239,845.760620,4
2,4d7e9b73-dd94-4fdc-9611-51ec641b8e19,2023-04-03,0.0,1.0,0.0,1,1,0.0,1.0,0.0,...,0,0,5,850.0,722.0,810.794650,5,0.010144,989.855778,5
3,2e3c32e2-1f1b-4d0f-a7c2-8dcdb11c6317,2023-04-03,0.0,0.0,0.0,1,0,3.0,19.0,21.0,...,0,0,5,795.0,721.0,908.033142,5,0.348148,651.852262,3
4,57948378-e103-4f19-972e-a911082a16c2,2023-04-03,0.0,0.0,0.0,1,0,17.0,2.0,19.0,...,0,0,3,551.0,652.0,712.696282,5,0.434676,565.323852,3


In [11]:
dff = df0.copy()

In [12]:
len(dff.loc[dff['emi_no'].isin([2,3,4,5]),'business_id'].unique())

1381

In [ ]:
# dff.to_pickle('/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/data/dataset_scored_original.pkl')

In [13]:
tdf = dff[['business_id','emi_no','redec_proba','redec_score','redec_bin']].merge(t,left_on=['business_id','emi_no'],right_on=['business_id','emi_no'])

In [14]:
tdf.head()

,business_id,emi_no,redec_proba,redec_score,redec_bin,distinct_debit_txns_1m,median_amount_debited_2m,ratio_debit_credit_2m,ratio_ach_credit_freq_total_credit_1m,ratio_ach_debit_credit_freq_1m,median_running_balance_2nd_m,ratio_revenue_1m_2m,distinct_cashins_1m_prev,ratio_amt_drawn_mom,ratio_amt_cashin_mom
0,c616b39b-aef7-4156-973a-08278aa74536,1,0.031045,968.955142,5,36.0,42.995,1.570221,0.000000,1.000000,21236.060,12494.280000,2.0,0.357015,2884.000000
1,cc861415-fd34-4654-8b97-dab0794938dc,1,0.154239,845.760620,4,62.0,101.750,0.987481,0.666667,2.800000,2122.430,0.806541,2.0,0.519896,2433.330000
2,4d7e9b73-dd94-4fdc-9611-51ec641b8e19,1,0.010144,989.855778,5,17.0,828.440,1.124376,0.000000,1.000000,5080.880,0.159984,2.0,0.000000,2196.830000
3,2e3c32e2-1f1b-4d0f-a7c2-8dcdb11c6317,1,0.348148,651.852262,3,72.0,19.000,1.142170,0.677419,0.863636,3749.750,1.101293,5.0,0.000000,0.949472
4,57948378-e103-4f19-972e-a911082a16c2,1,0.434676,565.323852,3,9.0,0.000,0.975947,0.950000,0.100000,1077.775,6.283541,1.0,0.000000,363.000000


In [15]:
# tdf.head(500).to_csv('/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/data/testing_data.csv')

In [ ]:
pd.crosstab(dff['rs2_bin'],dff['DPD_plus_15_next']).to_clipboard()

In [ ]:
pd.crosstab(dff['rs1_bin'],dff['DPD_plus_15_next']).to_clipboard()

In [ ]:
pd.crosstab(dff['redec_bin'],dff['DPD_plus_15_next']).to_clipboard()

In [ ]:
dff[dff['DPD_plus_15_next']==1].shape

In [ ]:
dff.loc[dff['DPD_plus_15_next']==1,'emi_no'].value_counts()

In [ ]:
pd.crosstab(dff['emi_no'],dff['DPD_plus_15_next']).to_clipboard()

In [ ]:
temp = dff[dff['emi_no'].isin([2,3,4,5])]

In [ ]:
temp.shape

In [ ]:
pd.crosstab(temp['rs2_bin'],temp['DPD_plus_15_next']).to_clipboard()

In [ ]:
tdf = dff.groupby('redec_bin',as_index=False)['DPD_plus_15_next'].agg(['sum','count'])
tdf['volume%'] = np.round(tdf['count']*100/len(dff),1)
tdf['default_rate'] = np.round(tdf['sum']*100/tdf['count'],1)
tdf.to_clipboard()

In [ ]:
tdf = dff.groupby('rs2_bin',as_index=False)['DPD_plus_15_next'].agg(['sum','count'])
tdf['volume%'] = np.round(tdf['count']*100/len(dff),1)
tdf['default_rate'] = np.round(tdf['sum']*100/tdf['count'],1)
tdf.to_clipboard()

In [ ]:
tdf = dff.groupby('rs1_bin',as_index=False)['DPD_plus_15_next'].agg(['sum','count'])
tdf['volume%'] = np.round(tdf['count']*100/len(dff),1)
tdf['default_rate'] = np.round(tdf['sum']*100/tdf['count'],1)
tdf.to_clipboard()

In [ ]:
tdf = dff.groupby(['rs2_bin','redec_bin'],as_index=False)['DPD_plus_15_next'].agg(['sum','count'])
tdf['volume%'] = np.round(tdf['count']*100/len(dff),1)
tdf['default_rate'] = np.round(tdf['sum']*100/tdf['count'],1)
tdf.to_clipboard()

In [ ]:
tdf = dff[dff['emi_no'].isin([2,3,4,5])].groupby(['redec_bin','emi_no'],as_index=False)['DPD_plus_15_next'].agg(['sum','count'])
tdf['volume%'] = np.round(tdf['count']*100/len(dff),1)
tdf['default_rate'] = np.round(tdf['sum']*100/tdf['count'],1)
tdf.to_clipboard()

In [ ]:
tdf = dff.groupby(['redec_bin','DPD_plus_3'],as_index=False)['DPD_plus_15_next'].agg(['sum','count'])
tdf['volume%'] = np.round(tdf['count']*100/len(dff),1)
tdf['default_rate'] = np.round(tdf['sum']*100/tdf['count'],1)
tdf

#### Binning from new dataset

In [ ]:
def new_buckets(x):
    if x > 0.6:
        return 1
    if x > 0.46:
        return 2
    if x > 0.31:
        return 3
    if x > 0.08:
        return 4
    return 5

In [ ]:
# def new_buckets(x):
#     if x > 0.7:
#         return 1
#     if x > 0.48:
#         return 2
#     if x > 0.35:
#         return 3
#     if x > 0.1:
#         return 4
#     return 5

In [ ]:
df0 = dff.copy()

In [ ]:
df0['rds_bin'] = df0['redec_proba'].apply(lambda x: new_buckets(x))

In [ ]:
ttdf = df0.groupby('rds_bin',as_index=False)['DPD_plus_15_next'].agg(['sum','count'])
ttdf['volume%'] = np.round(ttdf['count']*100/len(df0),1)
ttdf['default_rate'] = np.round(ttdf['sum']*100/ttdf['count'],1)
ttdf

In [ ]:
ttdf.to_clipboard()

#### Saving

In [ ]:
# df0.to_pickle('/Users/shashankgupta/Documents/code/git_project/re_decisioning/data/scored_dataset.pkl')

In [ ]:
# from sqlalchemy.types import NVARCHAR
# # from conf.config import SQLQuery
# q = SQLQuery('snowflake')

# df0.to_sql(name='redec_scored_dataset',
#                  con=q.engine, 
#                  schema='prod_db.adhoc',
#                  if_exists='append', 
#                  index=False, 
#                  chunksize=16000, 
#                  method='multi',
#                  dtype={col_name: NVARCHAR for col_name in df0})

In [ ]:
df0['redec_bin'].value_counts()

In [ ]:
len(df0['business_id'].unique())

In [ ]:
df0.shape